# Frontier Models

This section uses AWS Bedrock to run inference tests using the Claude and Open AI models. We use LangChain and Python multi-threading to run the inference tasks and quickly loop through all of the prompts to generate our output.

In [1]:
import json
import boto3
import botocore
import datetime
import importlib
import prompts
from tqdm import tqdm

from prompts import SYSTEM_PROMPTS
from IPython.display import Markdown, display
import constructors.langchain_strategy as ls


In [2]:
importlib.reload(prompts)
importlib.reload(ls)

<module 'constructors.langchain_strategy' from '/project/constructors/langchain_strategy.py'>

In [3]:
# boto3 must be version 1.34.51 or higher
boto3.__version__

'1.38.8'

In [4]:
# Bedrock client initialise
config = botocore.config.Config(read_timeout=1000)
boto3_bedrock = boto3.client('bedrock-runtime',config=config)

## Strategy Construction for Frontier Models

In [5]:
frontier = ls.FrontierRun(run_name='Claude 3.7',
                       model_id='us.anthropic.claude-3-7-sonnet-20250219-v1:0',
                       dataset_id='dow_quarterly_ltm_v3.json',system_prompt=prompts.SYSTEM_PROMPTS['BASE_CLAUDE']['prompt'],boto3_bedrock=boto3_bedrock)

In [23]:
prompts = frontier.setup_prompts()

In [6]:
frontier.run()

100%|██████████| 5/5 [00:10<00:00,  2.05s/it]

unable to save file - you can access via self.output to cache your results from the backtest!
unable to save file - you can access via self.output to cache your results from the backtest!


## Initial Bedrock Test
This is an initial run of Bedrock with Anthropic to see how it responds to the financial analysis task as a single model run. The below code was used for the development of the strategy construction module for LangChain

In [7]:

# set the model
model = 'us.anthropic.claude-3-7-sonnet-20250219-v1:0'
# load prompts into memory
with open('Data/prompts.json', 'rb') as f:
    prompts = json.load(f)

In [ ]:
def setup_claude_request(prompt: str) -> str:
    
    native_request = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 5000,
        "temperature": 0.7,
        "messages": [
            {
                "role":"user",
                "content": [{"type":"text", "text":prompt}]
            }
        ]
    }
    
    return json.dumps(native_request)

In [ ]:
GUARDRAIL_ID = os.environ['BEDROCK_GUARDRAIL_ID']
GUARDRAIL_VERSION = os.environ['BEDROCK_GUARDRAIL_VERSION']

model_id = model


def invoke_model_claude(prompt: dict, model_id: str) -> str:
    """Invoke the model using Bedrock. This is specifically designed for Anthropic models"""
    accept = 'application/json'
    contentType = 'application/json'

    # set up the request
    request = setup_claude_request(prompt)
    try:
        
        response = boto3_bedrock.invoke_model(
            body=request, 
            modelId=model_id, 
            accept=accept, 
            contentType=contentType,
            trace = "ENABLED"
        )
        response_body = json.loads(response.get('body').read())
        return response_body.get('content')[0].get('text') #Anthropic response template
    
    except botocore.exceptions.ClientError as error:
        
        if error.response['Error']['Code'] == 'AccessDeniedException':
               print(f"\x1b[41m{error.response['Error']['Message']}\
                    \nTo troubeshoot this issue please refer to the following resources.\
                     \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                     \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
            
        else:
            raise error

In [ ]:
response = invoke_model_claude(prompt, model_id)

In [ ]:
display(Markdown(response))

## LangChain version
This is the langgraph version in prep for multi-agentic AI system

### Run Inference Loop for Dow Jones

In [7]:

from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence
from langchain_core.rate_limiters import InMemoryRateLimiter

from langchain_aws import ChatBedrock
from pydantic import BaseModel, Field

import concurrent.futures

In [7]:
# set up the LLM in Bedrock
rate_limiter = InMemoryRateLimiter(
    requests_per_second=50,
    check_every_n_seconds=1,
    max_bucket_size=10,
)

llm = ChatBedrock(
    client = boto3_bedrock,
    #model = "us.meta.llama3-1-70b-instruct-v1:0",
    model = 'us.anthropic.claude-3-7-sonnet-20250219-v1:0',
    temperature = 0.5,#0.7,
    max_tokens=4000,
    rate_limiter = rate_limiter
)

In [8]:
system_prompt = SYSTEM_PROMPTS['COT_EARN_CLAUDE']['prompt']
prompt_template = PromptTemplate.from_template(system_prompt)

In [9]:
def run_model(prompt: dict, llm: RunnableSequence) -> dict:
    prompt_in = prompt_template.format(financials=prompt['prompt'][1]['content'])
    output = llm.invoke(prompt_in)
    decision_dict = {
        'date': prompt['date'],
        'security': prompt['security'],
        'response': output
    }
    return decision_dict

In [10]:
#prompt_template.format(financials=prompts[0]['prompt'][1]['content'])
prompt_template

PromptTemplate(input_variables=['financials'], input_types={}, partial_variables={}, template='You are a financial analyst. Use the following income statement, balance sheet and cash flow to make a decision on if earnings will increase over the next financial period. Think step-by-step through the financial statement analysis workflow. Your report should have the following sections: 1. Analysis of current profitability, liquidity, solvency and efficiency ratios; 2. time-series analysis across the ratios; 3. Analysis of financial performance; 4. Stock Price analysis; 5. Decision Analysis looking at the positive and negative factors as well as the weighting in the final decision; 6. Final Decision. Make your decision only on the datasets. Explain your reasons in less than 250 words. Indicate the magnitude of the increase or decrease. Provide a confidence score for how confident you are of the decision. If you are not confident then lower the confidence score. {financials}')

In [12]:
run_model(prompts[0], llm)

{'date': '2020-02-06',
 'security': 'MMM UN Equity',
 'response': AIMessage(content="# Financial Analysis: Earnings Forecast\n\nBased on a thorough analysis of the provided financial statements, I will assess whether earnings are likely to increase in the next financial period.\n\n## Key Observations\n\n### Revenue and Profitability Trends\n- Revenue shows slight growth from t-1 to t (1.5% increase from $7.99B to $8.11B)\n- However, operating income declined significantly from $2.01B in t-1 to $1.33B in t (34% decrease)\n- Net income decreased from $1.58B in t-1 to $969M in t (38.8% decrease)\n- EPS declined from $2.72 (diluted) to $1.66 (39% decrease)\n\n### Cost Structure\n- Operating expenses increased substantially from $1.79B in t-1 to $2.46B in t (37.3% increase)\n- SG&A expenses rose from $1.46B to $1.94B (33.3% increase)\n- R&D expenses increased from $443M to $521M (17.6% increase)\n- Abnormal losses of $348M in period t compared to gains of $112M in t-1\n\n### Balance Sheet A

In [11]:
%%time
# This creates the initial response - run this in parallel with 100 executors
data = []
with concurrent.futures.ThreadPoolExecutor(max_workers=100) as executor:
    futures = [executor.submit(run_model, prompt, llm) for prompt in prompts]
    data = [f.result() for f in futures]

CPU times: user 6.83 s, sys: 652 ms, total: 7.49 s
Wall time: 6min 8s


In [12]:
len(data)

896

In [13]:
cleaned_output = [{'date': i['date'], 
             'security': i['security'], 
             'response': i['response'].content} 
            for i in data]

In [14]:
with open('Data/Results - Bedrock2.json', 'w') as f:
    json.dump(cleaned_output, f)

## Convert to JSON format
This is needed to run the trade analytics and compare to other LLMs

In [24]:
rate_limiter = InMemoryRateLimiter(
    requests_per_second=10,
    check_every_n_seconds=1,
    max_bucket_size=10,
)

llm = ChatBedrock(
    client = boto3_bedrock,
    #model = "us.meta.llama3-1-70b-instruct-v1:0",
    model = 'us.anthropic.claude-3-7-sonnet-20250219-v1:0',
    temperature = 0.5,#0.7,
    max_tokens=4000,
    rate_limiter = rate_limiter
)

In [48]:
# STEP 1: Set up the structured output
# class FinancialOutput(BaseModel):
#     """Answer to the users question along with justification"""
#     decision: str = Field(..., description="Final Decision: BUY, SELL or HOLD")
#     reason: str = Field(..., description="Summary of the final decision")
#     confidence: int = Field(..., description="How confident you are of the decision")

# with open('Results/Earnings/results - deepseek - cot - 2025-03-30 .828629.json', 'rb') as f:
#     raw_output = json.load(f)

class FinancialOutput(BaseModel):
    """Answer to the users question along with justification"""
    direction: str = Field(..., description="earnings will increase or decrease or stay flat")
    magnitude: str = Field(..., description="size of the increase or decrease")
    reason: str = Field(..., description="Summary of the final decision")
    confidence: str = Field(..., description="How confident you are of the decision")

In [42]:
cleaned_output = raw_output['results']

In [49]:
# STEP 2: set up the system prompt and set the structured output format
system_prompt = "You are an assistant to a financial analyst. You are responsible for formatting the documents that the analyst produces into a machine readable format. Use only the information provided in the context. Convert it into the structured output. Do not add anything to the analysts report and do not change the recommendation. Do not hallucinate. Find the investment decision. Find the conclusion. Add all of the wording of the thought process into the steps section. context: {context}"

prompt_template = PromptTemplate.from_template(system_prompt)
structured_llm = llm.with_structured_output(FinancialOutput)


In [50]:
# STEP 3: Set up the function to call in each thread
# def run_cleanup(prompt: dict, llm: RunnableSequence) -> dict:
#     thought_process = prompt['response']
#     prompt_in = prompt_template.format(context=thought_process)
#     response = llm.invoke(prompt_in)
#     final_dict = {
#         'date': prompt['date'],
#         'security': prompt['security'],
#         'response': {
#             'decision': response.decision,
#             'reason': response.reason,
#             'confidence': response.confidence,
#             'thought_process': thought_process
#         }
#     }
#     return final_dict

def run_cleanup(prompt: dict, llm: RunnableSequence) -> dict:
    thought_process = prompt['response']
    prompt_in = prompt_template.format(context=thought_process)
    response = llm.invoke(prompt_in)
    final_dict = {
        'date': prompt['date'],
        'security': prompt['security'],
        'response': {
            'analyst_direction': response.direction,
            'analyst_magnitude': response.magnitude,
            'reason': response.reason,
            'confidence': response.confidence,
            'thought_process': thought_process
        }
    }
    return final_dict

In [51]:
# STEP 4: 

responses = []
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(run_cleanup, prompt, structured_llm) for prompt in cleaned_output]
    responses = [f.result() for f in futures]

In [52]:
# # STEP 5: convert to the correct format
final_json = {
    'run_date': str(datetime.datetime.now()),
    'system_prompt': SYSTEM_PROMPTS['COT_EARN_CLAUDE']['prompt'],
    'dataset': 'data_quarterly_pit_indu_blended',
    'model': model,
    'results': responses
}

# final_json = {
#     'run_date': str(datetime.datetime.now()),
#     'system_prompt': raw_output['system_prompt'],
#     'dataset': raw_output['dataset'],
#     'model': raw_output['model'],
#     'results': responses
# }

In [53]:
# STEP 6: Save the results 
with open(f'Results/results - Deepseek - COT C -{str(datetime.datetime.now())}.json', 'w') as f:
    json.dump(final_json, f)